# Export to GeoJSON for Mapbox GL (v2, robust IDs)
Fixes:
- Feature `id` now accepts **strings** (no int() cast), avoiding ValueError on non-numeric IDs.
- You can **exclude large/text columns** from properties to keep files small.
- Safer JSON serialization for pandas/numpy types.


## 1) Setup

In [3]:
# %pip install pandas shapely
import json
from pathlib import Path

import pandas as pd
from shapely.geometry import MultiPoint

try:
    import numpy as np
except Exception:
    np = None

try:
    import geopandas as gpd
except Exception:
    gpd = None


## 2) Load data

In [4]:
# Option A: Load from CSV (set csv_path). If df exists already, skip this.
csv_path = 'Adam F. - 2025 Miami Brokerage and Broker Rankings - Agents_Brokers.csv'  # e.g., 'data/listings.csv'
if csv_path:
    df = pd.read_csv(csv_path)

# Basic validation
required_cols = ['latitude','longitude','Agent_Name','Brokerage_Firm']
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

df = df.dropna(subset=['latitude','longitude']).copy()
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
df = df.dropna(subset=['latitude','longitude'])
df = df[(df['latitude'].between(-90,90)) & (df['longitude'].between(-180,180))].copy()

print("Rows after cleaning:", len(df))

Rows after cleaning: 26844


## 3) Helper functions (robust IDs & optional property filtering)

In [5]:
# Exclude properties you don't want to ship in the GeoJSON (helps keep files small).
# Example: large text columns like notes/descriptions.
EXCLUDE_PROPS = set(['TRD_note'])  # add or remove as needed

def _to_native(v):
    """Convert numpy/pandas scalars to native Python types; fallback to str for unknowns."""
    try:
        # Handle pandas NA
        import pandas as pd
        if pd.isna(v):
            return None
    except Exception:
        pass
    if 'numpy' in str(type(v)).lower():
        try:
            return v.item()
        except Exception:
            return str(v)
    return v

def to_point_feature(row, id_field='Unique_ID', use_id=True):
    # [lon, lat]
    coords = [float(row['longitude']), float(row['latitude'])]

    # Build properties excluding lat/lon and optional EXCLUDE_PROPS
    props_cols = [c for c in row.index if c not in ('latitude','longitude') and c not in EXCLUDE_PROPS]
    props = {c: _to_native(row[c]) for c in props_cols}

    feature = {
        "type": "Feature",
        "properties": props,
        "geometry": {"type": "Point", "coordinates": coords}
    }

    # Use string-safe IDs (optional)
    if use_id and id_field in row and pd.notna(row[id_field]):
        feature["id"] = str(row[id_field])

    return feature

def export_points_geojson(df, out_path, id_field='Unique_ID', use_id=True):
    features = [to_point_feature(r, id_field=id_field, use_id=use_id) for _, r in df.iterrows()]
    fc = {"type": "FeatureCollection", "features": features}
    Path(out_path).write_text(json.dumps(fc, ensure_ascii=False))
    print("Wrote", out_path, f"({len(features)} features)")

def convex_hull_feature(key, df_group, group_label, simplify_tol=None):
    pts = list(zip(df_group['longitude'].astype(float), df_group['latitude'].astype(float)))
    if len(pts) < 3:
        return None
    mp = MultiPoint(pts)
    hull = mp.convex_hull
    if hull.geom_type != 'Polygon':
        return None
    if simplify_tol:
        hull = hull.simplify(simplify_tol, preserve_topology=True)
        if hull.geom_type != 'Polygon':
            return None
    coords = list(hull.exterior.coords)  # (lon, lat)
    if coords[0] != coords[-1]:
        coords.append(coords[0])
    props = {group_label: key, "count": int(len(df_group))}
    return {
        "type": "Feature",
        "properties": props,
        "geometry": {"type": "Polygon", "coordinates": [coords]}
    }

def export_hulls_geojson(df, group_col, out_path, simplify_tol=None, min_points=3):
    features = []
    for key, g in df.groupby(group_col):
        if len(g) < min_points:
            continue
        f = convex_hull_feature(key, g, group_label=group_col, simplify_tol=simplify_tol)
        if f:
            features.append(f)
    fc = {"type": "FeatureCollection", "features": features}
    Path(out_path).write_text(json.dumps(fc, ensure_ascii=False))
    print("Wrote", out_path, f"({len(features)} hulls)")

## 4) Export GeoJSON files

In [6]:
# Tune for very large datasets
SIMPLIFY_TOL = 0.0008   # ~80m at equator
MIN_GROUP_POINTS = 5

# Points
export_points_geojson(df, 'agents_points.geojson', id_field='Unique_ID', use_id=True)
# Optionally rename Brokerage for clarity in downstream tools
df_b = df.rename(columns={'Brokerage_Firm':'Brokerage'})
export_points_geojson(df_b, 'brokerages_points.geojson', id_field='Unique_ID', use_id=True)

# Hulls
export_hulls_geojson(df, 'Agent_Name', 'agents_hulls.geojson', simplify_tol=SIMPLIFY_TOL, min_points=MIN_GROUP_POINTS)
export_hulls_geojson(df, 'Brokerage_Firm', 'brokerages_hulls.geojson', simplify_tol=SIMPLIFY_TOL, min_points=MIN_GROUP_POINTS)

Wrote agents_points.geojson (26844 features)
Wrote brokerages_points.geojson (26844 features)
Wrote agents_hulls.geojson (1156 hulls)
Wrote brokerages_hulls.geojson (715 hulls)


## 5) Minimal Mapbox GL JS template (same as before)

In [7]:
from pathlib import Path

html = '''<!doctype html>
<html>
<head>
  <meta charset="utf-8" />
  <title>Zones of Prevalence — Mapbox</title>
  <meta name="viewport" content="initial-scale=1,maximum-scale=1,user-scalable=no" />
  <script src="https://api.mapbox.com/mapbox-gl-js/v2.15.0/mapbox-gl.js"></script>
  <link href="https://api.mapbox.com/mapbox-gl-js/v2.15.0/mapbox-gl.css" rel="stylesheet" />
  <style>
    body { margin: 0; padding: 0; }
    #map { position: absolute; top: 0; bottom: 0; width: 100%; }
    .map-overlay {
      position: absolute; top: 10px; left: 10px; background: white; padding: 8px 10px;
      font: 12px/1.4 system-ui, -apple-system, Segoe UI, Roboto, Helvetica, Arial;
      border-radius: 6px; box-shadow: 0 1px 6px rgba(0,0,0,0.2);
    }
    .toggle { display: block; margin: 4px 0; }
  </style>
</head>
<body>
<div id="map"></div>
<div class="map-overlay">
  <strong>Layers</strong>
  <label class="toggle"><input id="agents-hulls" type="checkbox" checked> Agent Zones</label>
  <label class="toggle"><input id="brokerages-hulls" type="checkbox" checked> Brokerage Zones</label>
  <label class="toggle"><input id="agents-points" type="checkbox" checked> Agent Points</label>
  <label class="toggle"><input id="brokerages-points" type="checkbox" checked> Brokerage Points</label>
</div>

<script>
  mapboxgl.accessToken = 'YOUR_MAPBOX_ACCESS_TOKEN_HERE';

  const map = new mapboxgl.Map({
    container: 'map',
    style: 'mapbox://styles/mapbox/light-v11',
    center: [-73.98, 40.75],
    zoom: 10
  });

  map.on('load', () => {
    map.addSource('agents-hulls', { type: 'geojson', data: 'agents_hulls.geojson' });
    map.addSource('brokerages-hulls', { type: 'geojson', data: 'brokerages_hulls.geojson' });
    map.addSource('agents-points', { type: 'geojson', data: 'agents_points.geojson' });
    map.addSource('brokerages-points', { type: 'geojson', data: 'brokerages_points.geojson' });

    map.addLayer({ id: 'agents-hulls-fill', type: 'fill', source: 'agents-hulls', paint: { 'fill-opacity': 0.25 } });
    map.addLayer({ id: 'agents-hulls-outline', type: 'line', source: 'agents-hulls', paint: { 'line-width': 1 } });

    map.addLayer({ id: 'brokerages-hulls-fill', type: 'fill', source: 'brokerages-hulls', paint: { 'fill-opacity': 0.25 } });
    map.addLayer({ id: 'brokerages-hulls-outline', type: 'line', source: 'brokerages-hulls', paint: { 'line-width': 1 } });

    map.addLayer({ id: 'agents-points-layer', type: 'circle', source: 'agents-points', paint: { 'circle-radius': 3 } });
    map.addLayer({ id: 'brokerages-points-layer', type: 'circle', source: 'brokerages-points', paint: { 'circle-radius': 3 } });

    const toggles = [
      ['agents-hulls', ['agents-hulls-fill','agents-hulls-outline']],
      ['brokerages-hulls', ['brokerages-hulls-fill','brokerages-hulls-outline']],
      ['agents-points', ['agents-points-layer']],
      ['brokerages-points', ['brokerages-points-layer']]
    ];
    for (const [checkboxId, layerIds] of toggles) {
      const el = document.getElementById(checkboxId);
      el.addEventListener('change', () => {
        for (const lid of layerIds) {
          map.setLayoutProperty(lid, 'visibility', el.checked ? 'visible' : 'none');
        }
      });
    }
  });
</script>
</body>
</html>'''
Path('index.html').write_text(html)
print("Wrote index.html")

index.html


In [10]:
map_url = "https://trd-digital.github.io/trd-news-interactive-maps/sofla_2025_agentBroker_zones/"
print(map_url)

https://trd-digital.github.io/trd-news-interactive-maps/sofla_2025_agentBroker_zones/
